In [ ]:
# -*- coding: utf-8 -*-
"""rise_spatial_multiplicative_norm_zero_cineca.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1-4UndkKCTVbDRQZql3km54NZicEG05gb

### ***Cineca***
"""

import tensorflow as tf
from tensorflow.keras import layers, activations, callbacks, models
import numpy as np
import pickle
import os
from keras.models import load_model
from skimage.transform import resize
from tqdm import tqdm
import copy
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

"""
##### ***Data & Black-Box***

"""

# IMPORTO I DATI PER VOTTIGNASCO
import os

# Ottieni il percorso effettivo da una variabile d'ambiente
work_path = os.environ['WORK']  # Ottieni il valore della variabile d'ambiente WORK
v_test_OHE_path = os.path.join(work_path, "Water_Resources/rise-video/data/Vottignasco/Vottignasco_00425010001_test_month_OHE.npy")
v_test_image_path = os.path.join(work_path, "Water_Resources/rise-video/data/Vottignasco/Vottignasco_00425010001_test_normalized_image_sequences.npy")
v_test_target_dates_path = os.path.join(work_path, "Water_Resources/rise-video/data/Vottignasco/Vottignasco_00425010001_test_target_dates.npy")

# Carica l'array numpy dai file
vottignasco_test_OHE    = np.load(v_test_OHE_path)
vottignasco_test_image  = np.load(v_test_image_path)
vottignasco_test_dates  = np.load(v_test_target_dates_path)


print(len(vottignasco_test_dates))
print(len(vottignasco_test_image))
print(len(vottignasco_test_OHE))

#print(vottingasco_test_OHE[0], "\n")
#print(vottignasco_test_image[0][0], "\n")

# """##### ***Black Boxes***""

# Se vuoi abilitare il dropout a runtime
mc_dropout = True

# Definizione della classe personalizzata doprout_custom
class doprout_custom(tf.keras.layers.SpatialDropout1D):
    def call(self, inputs, training=None):
        if mc_dropout:
            return super().call(inputs, training=True)
        else:
            return super().call(inputs, training=False)

# Percorso della directory su Cineca
base_dir = os.path.join(os.environ['WORK'], "Water_Resources/rise-video/trained_models/seq2val/Vottignasco")
lstm_suffix = 'time_dist_LSTM'

vott_lstm_models = []

def extract_index(filename):
    """Funzione per estrarre l'indice finale dal nome del file."""
    return int(filename.split('_LSTM_')[-1].split('.')[0])

# Trova tutti i file .keras nella cartella e li aggiunge alla lista
for filename in os.listdir(base_dir):
    if lstm_suffix in filename and filename.endswith(".keras"):
        vott_lstm_models.append(os.path.join(base_dir, filename))

# Ordina i modelli in base all'indice finale
vott_lstm_models = sorted(vott_lstm_models, key=lambda x: extract_index(os.path.basename(x)))

# Lista per i modelli caricati
vott_lstm_models_loaded = []

for i, model_lstm_path in enumerate(vott_lstm_models[:10]):  # Prendo i primi 10 modelli ordinati
    #print(f"Caricamento del modello LSTM {i+1}: {model_lstm_path}")

    # Carico il modello con la classe custom
    model = load_model(model_lstm_path, custom_objects={"doprout_custom": doprout_custom})

    # Aggiungo il modello alla lista
    vott_lstm_models_loaded.append(model)

print(vott_lstm_models_loaded)

2025-03-05 22:49:18.495819: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-05 22:49:20.249805: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-05 22:49:25.851423: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-05 22:49:25.855371: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-05 22:49:26.559661: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

105
105
105
[<keras.src.engine.functional.Functional object at 0x7f15b71e6650>, <keras.src.engine.functional.Functional object at 0x7f15b07d31c0>, <keras.src.engine.functional.Functional object at 0x7f15b058fd90>, <keras.src.engine.functional.Functional object at 0x7f15b060e3b0>, <keras.src.engine.functional.Functional object at 0x7f15b04864a0>, <keras.src.engine.functional.Functional object at 0x7f15b04dd210>, <keras.src.engine.functional.Functional object at 0x7f15b0516a10>, <keras.src.engine.functional.Functional object at 0x7f15b037f310>, <keras.src.engine.functional.Functional object at 0x7f15b02351e0>, <keras.src.engine.functional.Functional object at 0x7f15b0271a80>]


In [ ]:
import random
import cv2
import numpy as np
import matplotlib.pyplot as plt

def generate_rise_masks_2d(N, input_size, seed, **kwargs):
    """
    Genera N maschere RISE per un'immagine di dimensioni HxW.

    Parametri:
    - N: numero di maschere
    - input_size: (H,W) dimensioni dell'immagine originale
    - h, w: dimensioni delle maschere a bassa risoluzione
    - p: probabilità di attivazione dei pixel nella maschera binaria iniziale

    Ritorna:
    - masks: array di shape (N, H, W) contenente le maschere normalizzate.
    """

    h  = kwargs.get("h", 2)
    w = kwargs.get("w", 2)
    p = kwargs.get("p", 0.5)

    np.random.seed(seed)

    masks = []
    H,W = input_size
    CH, CW = H // h, W // w  # Fattore di upscaling

    for _ in range(N):
        # 1. Generazione della maschera binaria iniziale (h x w)
        small_mask = np.random.rand(h, w) < p

        up_size_h = (h+1) * CH
        up_size_w = (w+1) * CW

        # 2. Upsampling bilineare alla dimensione (H + CH, W + CW
        upsampled_mask = cv2.resize(small_mask.astype(np.float32),
                                    (up_size_w, up_size_h), interpolation=cv2.INTER_LINEAR)
        
        #print(upsampled_mask.shape)
        
        # 3. Crop casuale della regione H x W
        x_offset = np.random.randint(0, (up_size_h - H) + 1)
        y_offset = np.random.randint(0, (up_size_w - W) + 1)
        final_mask = upsampled_mask[x_offset:x_offset + H, y_offset:y_offset + W] 

        #print(final_mask.shape)

        masks.append(final_mask)

    masks = np.array(masks)  # Converte la lista in array NumPy
    masks = masks[~(masks == 0).all(axis=(1, 2))]  # Filtro maschere vuote
    masks = masks[~(masks == 1.0).all(axis=(1, 2))]  # Filtro maschere con tutti 1.0


    return np.array(masks)

In [ ]:
# input_size = (5,8)
# h,w = 2,3
# p   = 0.5

# N = 100

# seed = 42

# masks = generate_rise_masks_2d(N, input_size, seed, h=h,w=w,p=p) 

# for nr_mask in range(0,len(masks),10):
#     plot_frame(masks[nr_mask], cmap="gray")

In [ ]:
# def plot_frame(frame, cmap='viridis', title="Frame"):
#     fig, ax = plt.subplots(figsize=(7, 5))  # Dimensioni della figura

#     ax.set_title(title)
#     im = ax.imshow(frame, cmap=cmap, alpha=1.0, origin="lower")  # Oggetto mappable

#     # Crea una colorbar della stessa altezza della figura
#     cbar = fig.colorbar(im, ax=ax, fraction=0.030, pad=0.04)

#     plt.show()

In [ ]:
# # Scelte 
# 2,2
# 2,3
# 2,4

In [76]:
def multiplicative_uniform_noise_onechannel(images, masks, channel, **kwargs):
    std_zero_value = kwargs.get("std_zero_value", -0.6486319166678826)

    masked = []

    # Itero su tutte le N maschere generate
    for mask in masks:
        masked_images = copy.deepcopy(images)  # Copia profonda delle immagini originali

        # Applica la perturbazione solo al canale specificato
        masked_images[..., channel] = (
            masked_images[..., channel] * mask + (1 - mask) * std_zero_value)

        masked.append(masked_images)

    return masked

def ensemble_predict(models, images, x3_exp, batch_size=1000):
    # Assicuriamoci che images sia una lista
    if not isinstance(images, list):
        images = [images]

    len_x3 = len(images)

    # Convertiamo x3_exp in un tensore replicato per ogni immagine
    x3_exp_tensor = tf.convert_to_tensor(x3_exp, dtype=tf.float32)

    # Lista per raccogliere le predizioni finali
    final_preds = []

    # Processamento a batch
    for i in range(0, len_x3, batch_size):
        batch_images = images[i:i + batch_size]
        batch_len = len(batch_images)

        # Conversione batch in tensori
        Y_test = tf.stack([tf.convert_to_tensor(img, dtype=tf.float32) for img in batch_images])
        Y_test_x3 = tf.tile(tf.expand_dims(x3_exp_tensor, axis=0), [batch_len, 1, 1])

        # Raccoglie le predizioni di tutti i modelli per il batch corrente
        batch_preds = []

        for model in models:
            preds = model.predict([Y_test, Y_test_x3], verbose=0)
            batch_preds.append(preds)

        # Converte le predizioni del batch in un tensore e calcola la media
        batch_preds_tensor = tf.stack(batch_preds)
        mean_batch_preds = tf.reduce_mean(batch_preds_tensor, axis=0)

        # Aggiunge le predizioni del batch alla lista finale
        final_preds.extend(mean_batch_preds.numpy())

    return np.array(final_preds)

"""#### ***Saliency Map***"""

# Modifica della funzione per calcolare la mappa di salienza introducendo anche -> E[M] cioè il valore atteso delle Maschere

def calculate_saliency_map_ev_masks(weights_array, masks):
    """
    Calcola la mappa di salienza media data una serie di predizioni e maschere.

    :param weights_array: Array di predizioni (pesi delle maschere).
    :param masks: Array di maschere (numero di maschere x dimensioni maschera).
    :return: Mappa di salienza media.
    """
    sal = []

    H, W = masks.shape[1], masks.shape[2]
    for j in range(len(masks)):
        sal_j = weights_array[j] * np.abs(masks[j])
        sal.append(sal_j.reshape(-1, H, W))

    # Rimuovere le dimensioni extra per fare np.mean lungo axis=0. masks ha shape (N,5,8,1)
    masks_squeezed = np.squeeze(np.abs(masks))
    # Ora calcola la media lungo l'asse 0
    ev_masks = np.mean(masks_squeezed, axis=0)

    sal = (1/ev_masks) * np.mean(sal, axis=0)  # aggiunta della frazione 1/valore_atteso(maschere)

    return np.squeeze(sal)

"""#### ***Spatial-RISE: Framework***"""

# Funzione sigmoide
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def rise_spatial_explain_sigmoide(nr_instance, data_test_image, data_test_OHE, models, channel,
                                  N, generate_masks_fn, seed, perturb_instance_fn, calculate_saliency_map_fn, **kwargs):
  print(f"############################### RISE-Spatial on Instance #{nr_instance} ###########################")
  instance    = copy.deepcopy(data_test_image[nr_instance])
  x3_instance = copy.deepcopy(data_test_OHE[nr_instance])

  input_size = (instance.shape[1], instance.shape[2])

  masks = generate_masks_fn(N, input_size, seed, **kwargs)
  perturbed_instances = perturb_instance_fn(instance, masks, channel)

  # Predizione su Istanza Originale
  pred_original = ensemble_predict(models, instance, x3_instance)
  # Predizioni su Istanze Perturbate
  preds_masked = ensemble_predict(models, perturbed_instances, x3_instance)

  # Differenza tra predizione originale e perturbata
  diff_pred = [abs(pred_original - pred_masked) for pred_masked in preds_masked]
  weights_array = np.concatenate(diff_pred, axis=0)

  # Standardizzazione (z-score normalization)
  mean = np.mean(weights_array)
  std = np.std(weights_array)
  standardized_weights = (weights_array - mean) / std
  # Applicare la sigmoide ai pesi standardizzati
  normalized_weights_std = sigmoid(standardized_weights)

  # Calcolo della mappa di salienza
  saliency_map_i = calculate_saliency_map_fn(1-normalized_weights_std, masks)
  print(f"############### Processo completato. Mappa di salienza generata per Istanza #{nr_instance} ###############")
  return np.squeeze(saliency_map_i)


In [ ]:
"""#### ***Evaluation Metrics***"""


def plot_insertion_curve(total_errors, auc, title="Insertion Metric Curve"):
    """
    Plotta la curva di metrica di insertion con l'errore medio quadratico.

    :param total_errors: Lista dei valori dell'errore per ogni frazione di pixel inseriti.
    :param auc: Valore dell'Area Under Curve (AUC) calcolato.
    :param title: Titolo del grafico (default: "Insertion Metric Curve").
    """

    # Nuovo asse X normalizzato tra 0 e 1
    x = np.linspace(0, 1, len(total_errors))

    # Plot della curva dell'errore e dell'area sotto la curva (AUC)
    plt.figure(figsize=(7, 5))
    plt.plot(x, total_errors, marker='o', linestyle='-', color='blue')

    # Pallini blu sui punti della curva
    plt.scatter(x, total_errors, color='blue', zorder=3)

    # Area sotto la curva
    plt.fill_between(x, total_errors, color='skyblue', alpha=0.4)

    # Testo "Error curve" in alto a dx con font più piccolo
    plt.legend(['Error curve'], loc='upper right', fontsize=9)

    # Testo AUC appena sotto "Error curve"
    plt.text(x[-1] - 0.02, max(total_errors) * 0.9,
             f'AUC: {auc:.2f}',
             horizontalalignment='right',
             fontsize=8,
             bbox=dict(facecolor='white', alpha=0.5))

    # Etichette assi
    plt.xlabel('Fraction of pixels inserted')
    plt.ylabel('Mean Squared Error')

    # Titolo del grafico
    plt.title(title)

    # Mostra il grafico
    plt.show()


def plot_deletion_curve(total_errors, auc, title="Deletion Metric Curve"):
    """
    Plotta la curva della metrica di deletion con l'errore medio quadratico.

    :param total_errors: Lista dei valori dell'errore per ogni frazione di pixel rimossi.
    :param auc: Valore dell'Area Under Curve (AUC) calcolato.
    :param title: Titolo del grafico (default: "Deletion Metric Curve").
    """

    # Normalizzazione dell'asse X tra 0 e 1
    x = np.linspace(0, 1, len(total_errors))

    # Creazione del plot
    plt.figure(figsize=(7,5))
    plt.plot(x, total_errors, marker='o', linestyle='-', color='red')

    # Pallini rossi sui punti della curva
    plt.scatter(x, total_errors, color='red', zorder=3)

    # Area sotto la curva
    plt.fill_between(x, total_errors, color='lightcoral', alpha=0.4)

    # Testo "Error curve" in alto a sx con font più piccolo
    plt.legend(['Error curve'], loc='upper left', fontsize=9)

    # Testo AUC appena sotto "Error curve"
    plt.text(x[0] + 0.01, max(total_errors) * 0.88,
             f'AUC: {auc:.2f}',
             horizontalalignment='left',
             fontsize=8,
             bbox=dict(facecolor='white', alpha=0.5))

    # Etichette degli assi
    plt.xlabel('Fraction of pixels removed')
    plt.ylabel('Mean Squared Error')

    # Titolo del grafico
    plt.title(title)

    # Mostra il grafico
    plt.show()


def calculate_auc(x, y):
    """
    Calcola l'area sotto la curva (AUC) utilizzando il metodo del trapezio.

    :param x: Valori dell'asse x (frazione dei pixel/frame inseriti).
    :param y: Valori dell'asse y (errori calcolati).
    :return: Area sotto la curva.
    """
    return np.trapz(y, x)


# Restituisce n-esimo percentile dei pixel più importanti delle mappa di salienza data in input
def get_top_n_pixels(saliency_map, n):
    # Appiattisci la mappa di salienza
    flat_saliency = saliency_map.flatten()
    # Ordina gli indici degli elementi in ordine decrescente di salienza
    sorted_indices = np.argsort(flat_saliency)[::-1]

    # Calcola il numero di colonne della mappa di salienza
    num_cols = saliency_map.shape[1]

    top_pixels = []
    for i in range(n):
        idx = sorted_indices[i]
        row, col = divmod(idx, num_cols)
        top_pixels.append((row, col))

    return top_pixels

"""##### ***Insertion***"""

def update_instance_with_pixels(current_instance, original_instance, x,y):
    """
    Aggiorna l'immagine inserendo i pixel più importanti.

    :param current_instance: Istanza corrente.
    :param original_instance: Istanza originale.
    :param x: coordinata x del pixel da inserire
    :param y: coordinata y del pixel da inserire
    :return: Istanza aggiornata con il superpixel.
    """
    new_current_instance = current_instance.copy()
    new_current_instance[:, x, y, 0] = original_instance[:, x, y, 0]

    return new_current_instance


def insertion(model, original_instance, x3_instance, sorted_per_importance_pixels_index, initial_blurred_instance, original_prediction):
    """
    Calcola la metrica di inserimento per una spiegazione data.

    :param model: Black-box.
    :param original_instance: Istanza originale.
    :param sorted_per_importance_pixels_index: Lista di liste di tutti i superpixel per importanza
    :param initial_blurred_images: Immagine iniziale con tutti i pixel a zero.
    :return: Lista degli errori ad ogni passo di inserimento.
    """

    # Lista per memorizzare le istanze a cui aggiungo pixel mano a mano. Inizializzata con istanza iniziale blurrata
    insertion_images = [initial_blurred_instance]

    # Predizione sull'immagine iniziale (tutti i pixel a zero)
    I_prime = copy.deepcopy(initial_blurred_instance)

    # Aggiungere gradualmente i pixel (per ogni frame) più importanti. Ottengo una lista con tutte le img con i pixel aggiunti in maniera graduale
    for x,y in sorted_per_importance_pixels_index:
        I_prime = update_instance_with_pixels(I_prime, original_instance, x,y)
        insertion_images.append(I_prime)

    insertion_images = [img.astype(np.float32) for img in insertion_images]
    # Calcolo le predizioni sulle istanze a cui ho aggiunto i pixel in maniera graduale
    new_predictions = ensemble_predict(model, insertion_images, x3_instance)
    # Rispetto ad ogni suddetta predizione, calcolo il MSE rispetto la pred sull'istanza originaria (come da test-set). Ignora la prima che è sull'img blurrata originale
    errors = [mean_squared_error(original_prediction, masked_pred) for masked_pred in new_predictions[1:]]

    initial_error = mean_squared_error(original_prediction, new_predictions[0])
    print(f"Initial Prediction with Blurred Instance. Prediction: {new_predictions[0]}, error: {initial_error}")
    only_inserted_pixel_new_predictions = new_predictions[1:]

    for nr_pixel, error in enumerate(errors):
      print(f"Inserted Pixel: {sorted_per_importance_pixels_index[nr_pixel]}. Prediction: {only_inserted_pixel_new_predictions[nr_pixel]}, error: {error}")

    total_errors = [initial_error] + errors # Errore iniziale + errori su tutti i pixel inseriti

    # # Nuovo asse X
    x = np.linspace(0, 1, len(total_errors))
    # Calcolo dell'AUC con il nuovo asse x
    auc = calculate_auc(x, total_errors)
    print(f"Area under the curve (AUC): {auc}")
    return total_errors,auc


def plot_insertion_error_mean_curve(errors_insertion_all_dataset):
    # Calcolo dell'errore medio per ogni numero di superpixel inseriti
    mean_errors_for_insertion_vott = np.mean(errors_insertion_all_dataset, axis=0)

    # Array x per il numero di superpixel inseriti
    x = np.arange(0, len(mean_errors_for_insertion_vott))  # Array dinamico basato sulla lunghezza dei dati
    auc = calculate_auc(x, mean_errors_for_insertion_vott)

    # Creazione del grafico
    plt.plot(x, mean_errors_for_insertion_vott, label='Error Curve')

    # Pallini blu sui punti della curva
    plt.scatter(x, mean_errors_for_insertion_vott, color='blue', zorder=3)

    # Area sotto la curva (AUC)
    plt.fill_between(x, mean_errors_for_insertion_vott, color='skyblue', alpha=0.4)

    # Etichette degli assi
    plt.xlabel('Nr of SuperPixels inserted')
    plt.ylabel('Mean Squared Error')

    # Griglia e stile
    plt.grid(True, linestyle='--', alpha=0.6)

    # Titolo e legenda
    plt.title('Mean Insertion Metric Curve')
    plt.legend()

    # Visualizzazione del grafico
    plt.show()

    return auc,mean_errors_for_insertion_vott

"""##### ***Deletion***"""


def update_image_by_removing_pixels(current_instance, x, y, std_zero_value=-0.6486319166678826):
    """
    Aggiorna l'immagine rimuovendo i pixel x,y indicati.

    :param current_instance: istanza corrente.
    :param x: coordinata x del pixel da rimuovere
    :param y: coordinata y del pixel da rimuovere
    :return: Istanza aggiornata con x,y rimossi su tutti time-step.
    """
    new_instance = copy.deepcopy(current_instance)
    new_instance[:, x, y, 0] = std_zero_value # Imposta i pixel a zero normalizzato per Prec
    return new_instance

def deletion(models, original_instance, x3_instance, sorted_per_importance_pixels_index, original_prediction):
    """
    Calcola la metrica di rimozione per una spiegazione data.

    :param models: Lista di modelli pre-addestrati.
    :param original_instance: Immagine originale.
    :param x3_instance: Codifica one-hot per la previsione.
    :param sorted_per_importance_pixels_index: Indici dei pixel in ordine di importanza.
    :return: Lista degli errori, auc ad ogni passo di rimozione.
    """
    # Lista per memorizzare le img a cui elimino gradualmente i pixels (per ogni time-step)
    deletion_images = []

    # Inizializzazione
    I_prime = copy.deepcopy(original_instance)

    # Aggiungere gradualmente i pixel (per ogni frame) più importanti. Ottengo una lista con tutte le img con i pixel rimossi
    for x, y in sorted_per_importance_pixels_index:
        I_prime = update_image_by_removing_pixels(I_prime, x, y)
        deletion_images.append(I_prime)

    # Calcolo della predizione su tutte le img a cui ho rimosso gradualmente i pixel
    new_predictions = ensemble_predict(models, deletion_images, x3_instance)
    # Calcolo del mse rispetto la predizione originale
    errors = [mean_squared_error(original_prediction, masked_pred) for masked_pred in new_predictions]

    initial_error = 0.0
    print(f"Initial Prediction with Original Images, prediction: {original_prediction}, error: {initial_error}")
    for nr_pixel, error in enumerate(errors):
        print(f"Removed pixel {sorted_per_importance_pixels_index[nr_pixel]}, new prediction: {new_predictions[nr_pixel]}, error: {error}")

    total_errors = [initial_error] + errors  # Errore iniziale + errori su tutti i pixel rimossi

    # Normalizzare la frazione di pixel rimossi
    x = np.linspace(0, 1, len(total_errors))
    # Calcolo dell'AUC
    auc = calculate_auc(x, total_errors)

    print(f"Area under the curve (AUC): {auc}")
    return total_errors, auc

def plot_deletion_error_mean_curve(only_errors_deletion_vott):
    """
    Funzione che calcola la curva di errore media per la rimozione dei superpixel,
    calcola l'area sotto la curva (AUC), e visualizza il grafico con l'errore e l'AUC.

    :param only_errors_deletion_vott: Array di errori per numero di superpixel rimossi.
    :return: AUC e la media degli errori per i superpixel rimossi.
    """
    # Calcola l'errore medio per ogni numero di superpixel rimossi
    mean_errors_for_deletion_vott = np.mean(only_errors_deletion_vott, axis=0)

    # Array x per il numero di superpixel rimossi
    x = np.arange(0, len(mean_errors_for_deletion_vott))  # Array dinamico basato sulla lunghezza dei dati

    # Calcola l'AUC utilizzando il metodo del trapezio
    auc = calculate_auc(x, mean_errors_for_deletion_vott)

    # Creazione del grafico
    plt.plot(x, mean_errors_for_deletion_vott, label='Error Curve')

    # Pallini rossi sui punti della curva
    plt.scatter(x, mean_errors_for_deletion_vott, color='red', zorder=3)

    # Area sotto la curva (AUC)
    plt.fill_between(x, mean_errors_for_deletion_vott, color='lightcoral', alpha=0.4)

    # Etichette degli assi
    plt.xlabel('Nr of SuperPixels removed')
    plt.ylabel('Mean Squared Error')

    # Griglia e stile
    plt.grid(True, linestyle='--', alpha=0.6)

    # Titolo e legenda
    plt.title('Deletion Mean Metric Curve')
    plt.legend()

    # Visualizzazione del grafico
    plt.show()

    # Restituisci l'AUC e la media degli errori
    return auc, mean_errors_for_deletion_vott


In [ ]:
"""### ***Experiments***

##### Sigmoide
"""

channel_prec = 0
models = vott_lstm_models_loaded
seed = 42
T,H,W,C = (104,5,8,3)
std_zero_value = -0.6486319166678826

N = 1000

h_w_values = [(2,2),(2,3),(2,4)]
p_values = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

results_setups = []

len_test_set = len(vottignasco_test_image)

for h,w in h_w_values:
    #print(f"############################## Setup h,w: ({h},{w}) ##############################")
    for p in p_values:
        # Conserva tutte le sal_maps per tutto il Test-Set
        saliency_maps = np.zeros((len_test_set,H,W))
        # Errori e AUC Insertion/Deletion tutto il Test-Set
        errors_insertion_all_testset = np.zeros((len_test_set, H*W+1))
        auc_insertion_all_testset    = np.zeros((len_test_set, 1))
        errors_deletion_all_testset  = np.zeros((len_test_set, H*W+1))
        auc_deletion_all_testset     = np.zeros((len_test_set, 1))

        p_str = str(p).replace(".", "")
        param_combination = f"h{h}_w{w}_p{p_str}"
        print(f"############################## Setup - > parameters combination: {param_combination} ##############################")   
        
        for nr_instance,_ in enumerate(vottignasco_test_image):
            saliency_map_i = rise_spatial_explain_sigmoide(nr_instance, vottignasco_test_image, vottignasco_test_OHE, models, channel_prec,
                                                     N, generate_rise_masks_2d, seed, multiplicative_uniform_noise_onechannel, calculate_saliency_map_ev_masks, h=h, w=w, p=p)
            
            #plot_frame(saliency_map_i, cmap="PuBu")
            
            
            saliency_maps[nr_instance] = saliency_map_i

            instance    = copy.deepcopy(vottignasco_test_image[nr_instance])
            x3_instance = copy.deepcopy(vottignasco_test_OHE[nr_instance])

            # Insertion
            # Video blurrato da cui partire per l'Insertion. Tutti i pixel di Prec su std_zero_value
            initial_blurred_instance = copy.deepcopy(instance)
            initial_blurred_instance[:,:,:,channel_prec] = std_zero_value

            all_important_pixels = get_top_n_pixels(saliency_map_i, instance.shape[1]*instance.shape[2])

            original_instance = copy.deepcopy(instance)
            original_prediction = ensemble_predict(models, original_instance, x3_instance)
            print(f"Original Prediction: {original_prediction}")

            errors_insertion,auc_insertion = insertion(models, original_instance, x3_instance, all_important_pixels, initial_blurred_instance, original_prediction)
            print(f"Errors Insertion: {errors_insertion}")
            print(f"AUC Insertion: {auc_insertion}")

            for nr_error, error in enumerate(errors_insertion):
                errors_insertion_all_testset[nr_instance][nr_error] = error
            
            auc_insertion_all_testset[nr_instance] = auc_insertion

            # Deletion
            errors_deletion,auc_deletion = deletion(models, original_instance, x3_instance, all_important_pixels, original_prediction)
            print(f"Errors Deletion: {errors_deletion}")
            print(f"AUC Deletion: {auc_deletion}")

            for nr_error, error in enumerate(errors_deletion):
                errors_deletion_all_testset[nr_instance][nr_error] = error
            
            auc_deletion_all_testset[nr_instance] = auc_deletion

            print(f"#################################### END for all Instance in Test-Set for {param_combination} ####################################")

            result = {
                "saliency_maps": saliency_maps,
                "errors_insertion": errors_insertion_all_testset,
                "auc_insertion": auc_insertion_all_testset,
                "errors_deletion": errors_deletion_all_testset,
                "auc_deletion": auc_deletion_all_testset,
                "parameters_comb": param_combination
            }

            path_to_save_results = os.path.join(work_path, f"Water_Resources/rise-video/XAI/spatial/results/rise_original_multiplicative_norm_zero/results_setup_new_h_w_inv_sigmoide.pkl")

            # Salvataggio della lista results in un file pickle
            with open(path_to_save_results, 'wb') as f:
                pickle.dump(results_setups, f)

print("############################# END FOR ALL SETUPS ##########################################################################")







NameError: name 'vott_lstm_models_loaded' is not defined